In [0]:
Name = 'Abhishek Mandge '
mail_id ='abhishekmandge@gmail.com'
mob_No = 9370227033
print(f"""Incubyte_assignment 
    Name: {Name}
    mail_Id: {mail_id}
    Contact_No: {mob_No}""")

Incubyte_assignment 
    Name: Abhishek Mandge 
    mail_Id: abhishekmandge@gmail.com
    Contact_No: 9370227033


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, datediff, current_date, floor

In [0]:
%sql
DESCRIBE EXTENDED Table_New_Zealand;

col_name,data_type,comment
Customer_Name,string,null
Customer_ID,string,null
Customer_Open_Date,date,null
Last_Consulted_Date,date,null
Vaccination_Type,string,null
Doctor_Consulted,string,null
State,string,null
Country,string,null
Post_Code,int,null
Date_of_Birth,date,null


In [0]:
%sql
CREATE TABLE Table_Philippines (
    Customer_Name VARCHAR(255) NOT NULL,
    Customer_ID VARCHAR(18) NOT NULL,
    Open_Date DATE NOT NULL,
    Last_Consulted_Date DATE,
    Vaccination_Type CHAR(5),
    Doctor_Consulted VARCHAR(255),
    State CHAR(5),
    Country CHAR(5),
    Post_Code INT,
    DOB DATE,
    Is_Active CHAR(1),
    Age INT,
    Days_Since_Last_Consulted INT
);


In [0]:
%sql
CREATE TABLE Table_New_Zealand (
    Customer_Name VARCHAR(255) NOT NULL,
    Customer_ID VARCHAR(18) NOT NULL,
    Open_Date DATE NOT NULL,
    Last_Consulted_Date DATE,
    Vaccination_Type CHAR(5),
    Doctor_Consulted VARCHAR(255),
    State CHAR(5),
    Country CHAR(5),
    Post_Code INT,
    DOB DATE,
    Is_Active CHAR(1),
    Age INT,
    Days_Since_Last_Consulted INT
);


In [0]:
spark = SparkSession.builder.appName("CustomerDataETL").getOrCreate()

# Load the customer data from the file (created Mock data CSV file)
df = spark.read.csv("/FileStore/tables/incubyte/Hospital_MOCK_DATA.csv", header=True, inferSchema=True)

In [0]:
from pyspark.sql.functions import col

# Rename columns to replace invalid characters like spaces
df_cleaned = df.select([col(c).alias(c.replace(" ", "_").replace("(", "").replace(")", "")) for c in df.columns])

# Add derived columns: Age and Days_Since_Last_Consulted
df_transformed = df_cleaned.withColumn("Age", floor(datediff(current_date(), col("Date_of_Birth")) / 365)) \
                           .withColumn("Days_Since_Last_Consulted", datediff(current_date(), col("Last_Consulted_Date")))

df_deduplicated = df_transformed.orderBy(col("Last_Consulted_Date").desc()).dropDuplicates(["Customer_ID"])


# Filter customers based on country and load into country-specific tables
df_philippines = df_transformed.filter(col("Country") == "Philippines")
df_New_Zealand = df_transformed.filter(col("Country") == "New Zealand")


# Write the data to a country-specific table, forcing schema overwrite
df_philippines.write.mode("overwrite").option("overwriteSchema", "true").format("delta").saveAsTable("Table_Philippines")
df_New_Zealand.write.mode("overwrite").option("overwriteSchema", "true").format("delta").saveAsTable("Table_New_Zealand")



In [0]:
df_transformed.display(10)

Customer_Name,Customer_ID,Customer_Open_Date,Last_Consulted_Date,Vaccination_Type,Doctor_Consulted,State,Country,Post_Code,Date_of_Birth,Active_Customer,Age,Days_Since_Last_Consulted
Scott,18 characters,2024-07-13,2024-06-28,C,Beau,null,Philippines,92334,2024-05-28,false,0,115
Evin,18 characters,2023-11-15,2023-12-27,C,Norbie,null,Philippines,20790,2023-11-04,true,0,299
Frederico,18 characters,2023-12-30,2024-09-23,B,Sarah,null,Philippines,66378,2024-10-03,false,0,28
Zorana,18 characters,2024-04-12,2024-09-24,B,Toma,null,Philippines,80701,2024-05-31,false,0,27
Klaus,18 characters,2023-11-18,2024-05-13,C,Cullie,null,Philippines,50419,2024-07-05,false,0,161
Gabe,18 characters,2023-11-23,2024-07-25,B,Lars,null,Philippines,98833,2024-07-14,true,0,88
Valma,18 characters,2024-04-30,2024-07-21,A,Britta,null,Philippines,4567,2024-03-25,false,0,92
Sascha,18 characters,2024-06-23,2024-07-28,A,Brew,null,Philippines,79831,2024-10-07,true,0,85
Philippe,18 characters,2023-12-22,2024-03-07,D,Willdon,null,Philippines,74752,2024-06-02,false,0,228
Nikolos,18 characters,2024-09-04,2024-09-11,A,Irena,null,Philippines,24321,2024-07-11,true,0,40


In [0]:
# Validation: Check for missing mandatory fields
mandatory_columns = ['Customer_Name', 'Customer_ID', 'Open_Date']
df_validation = df_transformed.filter((col('Customer_Name').isNull()) | (col('Customer_ID').isNull()))

if df_validation.count() > 0:
    print("Validation failed: Missing mandatory fields")

# Validation: Ensure correct date format (we can add specific regex checks if needed)
df_transformed.select("Last_Consulted_Date", "Date_of_Birth").show()


+-------------------+-------------+
|Last_Consulted_Date|Date_of_Birth|
+-------------------+-------------+
|         2024-06-28|   2024-05-28|
|         2023-12-27|   2023-11-04|
|         2024-09-23|   2024-10-03|
|         2024-09-24|   2024-05-31|
|         2024-05-13|   2024-07-05|
|         2024-07-25|   2024-07-14|
|         2024-07-21|   2024-03-25|
|         2024-07-28|   2024-10-07|
|         2024-03-07|   2024-06-02|
|         2024-09-11|   2024-07-11|
|         2024-04-09|   2023-10-25|
|         2023-12-24|   2024-06-19|
|         2023-11-05|   2024-05-10|
|         2024-03-31|   2024-05-31|
|         2024-01-20|   2024-04-09|
|         2024-02-27|   2024-09-01|
|         2023-12-18|   2023-11-21|
|         2024-02-15|   2024-04-18|
|         2024-09-15|   2024-06-13|
|         2024-07-02|   2024-02-21|
+-------------------+-------------+
only showing top 20 rows



In [0]:
# Write data partitioned by Country and Open_Date
df_transformed.write.partitionBy("Country", "Customer_Open_Date").mode("overwrite").format("delta").save("/mnt/data/partitioned_customer_data")


In [0]:
# Upsert: Merge records based on Customer_ID and update the latest Last_Consulted_Date
from delta.tables import DeltaTable

# Load the existing country-specific table
delta_table = DeltaTable.forPath(spark, "/mnt/data/partitioned_customer_data")

# Merge the new data into the existing table
delta_table.alias("tgt").merge(
        df_deduplicated.alias("src"),
        "tgt.Customer_ID = src.Customer_ID").whenMatchedUpdateAll(
        condition="src.Last_Consulted_Date > tgt.Last_Consulted_Date").whenNotMatchedInsertAll().execute()
